# Heart Disease Prediction: A Supervised Learning Project

## 1. Problem Statement

Cardiovascular diseases (CVDs) are the number one cause of death globally. Early detection and management of heart disease can significantly improve patient outcomes. In this project, we will develop a supervised learning model to predict whether a patient has heart disease based on their clinical parameters.

**Objective**: To build and evaluate machine learning models that can accurately predict the presence of heart disease based on patient data.

**Dataset**: Heart Disease UCI dataset from Kaggle (https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset)

**Target Variable**: Presence of heart disease (binary classification: 0 = no disease, 1 = disease)

## 2. Importing Libraries and Dataset


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

In [ ]:
df = pd.read_csv('heart.csv')

df.head()

## 3. Dataset Understanding

The dataset contains the following features:

1. **age**: Age in years
2. **sex**: Sex (1 = male, 0 = female)
3. **cp**: Chest pain type (0-3)
   - 0: Typical angina
   - 1: Atypical angina
   - 2: Non-anginal pain
   - 3: Asymptomatic
4. **trestbps**: Resting blood pressure (in mm Hg)
5. **chol**: Serum cholesterol in mg/dl
6. **fbs**: Fasting blood sugar > 120 mg/dl (1 = true, 0 = false)
7. **restecg**: Resting electrocardiographic results (0-2)
   - 0: Normal
   - 1: Having ST-T wave abnormality
   - 2: Showing probable or definite left ventricular hypertrophy
8. **thalach**: Maximum heart rate achieved
9. **exang**: Exercise induced angina (1 = yes, 0 = no)
10. **oldpeak**: ST depression induced by exercise relative to rest
11. **slope**: The slope of the peak exercise ST segment (0-2)
    - 0: Upsloping
    - 1: Flat
    - 2: Downsloping
12. **ca**: Number of major vessels colored by fluoroscopy (0-4)
13. **thal**: Thalassemia (0-3)
    - 0: NULL
    - 1: Fixed defect
    - 2: Normal
    - 3: Reversible defect
14. **target**: Diagnosis of heart disease (0 = no disease, 1 = disease)

Let's examine the dataset further:


In [ ]:
print(f"Dataset dimensions: {df.shape[0]} rows and {df.shape[1]} columns")

print("\nData types and missing values:")
print(df.info())

print("\nStatistical summary:")
print(df.describe())

print("\nMissing values in each column:")
print(df.isnull().sum())

print("\nTarget distribution:")
print(df['target'].value_counts())
print(f"Percentage of patients with heart disease: {df['target'].mean() * 100:.2f}%")

## 4. Exploratory Data Analysis (EDA)

Let's start by investigating the relationships between our features and the target variable:

### 4.1 Distribution of Numerical Features


In [ ]:
plt.figure(figsize=(20, 15))

numerical_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

for i, feature in enumerate(numerical_features):
    plt.subplot(3, 2, i+1)
    sns.histplot(data=df, x=feature, hue='target', kde=True, bins=20, alpha=0.7)
    plt.title(f'Distribution of {feature} by Heart Disease', fontsize=14)
    plt.xlabel(feature, fontsize=12)
    plt.ylabel('Count', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20, 15))

for i, feature in enumerate(numerical_features):
    plt.subplot(3, 2, i+1)
    sns.boxplot(data=df, x='target', y=feature)
    plt.title(f'Boxplot of {feature} by Heart Disease', fontsize=14)
    plt.xlabel('Heart Disease (1=Yes, 0=No)', fontsize=12)
    plt.ylabel(feature, fontsize=12)

plt.tight_layout()
plt.show()

### 4.2 Analysis of Categorical Features


In [ ]:
plt.figure(figsize=(20, 20))

categorical_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

for i, feature in enumerate(categorical_features):
    plt.subplot(4, 2, i+1)
    sns.countplot(data=df, x=feature, hue='target')
    plt.title(f'Count of {feature} by Heart Disease', fontsize=14)
    plt.xlabel(feature, fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.legend(title='Heart Disease', labels=['No', 'Yes'])

plt.tight_layout()
plt.show()

### 4.3 Correlation Analysis


In [ ]:
corr_matrix = df.corr()

plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix of Heart Disease Dataset', fontsize=16)
plt.show()

In [ ]:
target_corr = corr_matrix['target'].sort_values(ascending=False)
print("Correlation with target variable (heart disease):")
print(target_corr)

plt.figure(figsize=(12, 8))
target_corr = target_corr.drop('target')
sns.barplot(x=target_corr.values, y=target_corr.index)
plt.title('Feature Correlation with Heart Disease', fontsize=16)
plt.xlabel('Correlation Coefficient', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.axvline(x=0, color='gray', linestyle='--')
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

### 4.4 Feature Pair Relationships


In [ ]:
important_features = ['cp', 'thalach', 'ca', 'thal', 'oldpeak', 'exang']

sns.pairplot(df[important_features + ['target']], hue='target', diag_kind='kde', height=2.5)
plt.suptitle('Pairwise Relationships between Important Features', y=1.02, fontsize=16)
plt.show()

### 4.5 Key Insights from EDA

Based on our exploratory data analysis, we can draw several insights:

1. **Age**: Patients with heart disease tend to be older on average.
2. **Sex**: Men (sex=1) appear to have a higher incidence of heart disease than women.
3. **Chest Pain (cp)**: Asymptomatic chest pain (cp=3) is strongly associated with heart disease.
4. **Thalach**: Patients with heart disease tend to achieve higher maximum heart rates.
5. **ST Depression (oldpeak)**: Higher oldpeak values are associated with heart disease.
6. **Exercise-induced angina (exang)**: Patients experiencing exercise-induced angina are more likely to have heart disease.
7. **Number of major vessels (ca)**: As the number of major vessels increases, the likelihood of heart disease decreases.
8. **Thalassemia (thal)**: Reversible defect (thal=3) is strongly associated with heart disease.

The correlation analysis shows that 'cp', 'thalach', 'slope', and 'ca' have strong correlations with the target variable, making them potentially important predictors.

## 5. Data Preprocessing

### 5.1 Feature Selection


In [ ]:
features = df.drop('target', axis=1)
target = df['target']

print(f"Selected features: {features.columns.tolist()}")

### 5.2 Train-Test Split


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, stratify=target)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")
print(f"Training target distribution: \n{pd.Series(y_train).value_counts(normalize=True)}")
print(f"Testing target distribution: \n{pd.Series(y_test).value_counts(normalize=True)}")

### 5.3 Feature Scaling


In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=features.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=features.columns)

print("Scaled training data:")
print(X_train_scaled_df.head())

## 6. Model Building and Evaluation

We'll implement and evaluate several classification models to find the best performer:

1. Logistic Regression
2. Decision Tree
3. Random Forest
4. Support Vector Machine (SVM)

### 6.1 Define Evaluation Metrics


In [ ]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"Model: {model_name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    print(f"5-Fold CV Accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
    
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Confusion Matrix - {model_name}', fontsize=14)
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.show()
    
    if y_pred_prob is not None:
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        auc = roc_auc_score(y_test, y_pred_prob)
        
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f'AUC = {auc:.4f}')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate', fontsize=12)
        plt.ylabel('True Positive Rate', fontsize=12)
        plt.title(f'ROC Curve - {model_name}', fontsize=14)
        plt.legend(loc='lower right')
        plt.grid(True, alpha=0.3)
        plt.show()
    
    return {
        'model': model,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'cv_accuracy': cv_scores.mean()
    }

### 6.2 Logistic Regression


In [ ]:
log_reg = LogisticRegression(max_iter=1000, random_state=42)

log_reg_results = evaluate_model(log_reg, X_train_scaled, X_test_scaled, y_train, y_test, "Logistic Regression")

### 6.3 Decision Tree


In [ ]:
dt = DecisionTreeClassifier(random_state=42)

dt_results = evaluate_model(dt, X_train_scaled, X_test_scaled, y_train, y_test, "Decision Tree")

### 6.4 Random Forest


In [ ]:
rf = RandomForestClassifier(random_state=42)

rf_results = evaluate_model(rf, X_train_scaled, X_test_scaled, y_train, y_test, "Random Forest")

### 6.5 Support Vector Machine


In [ ]:
svm = SVC(probability=True, random_state=42)

svm_results = evaluate_model(svm, X_train_scaled, X_test_scaled, y_train, y_test, "Support Vector Machine")

### 6.6 Model Comparison


In [ ]:
models = [log_reg_results, dt_results, rf_results, svm_results]
model_names = ["Logistic Regression", "Decision Tree", "Random Forest", "SVM"]

comparison_df = pd.DataFrame({
    'Model': model_names,
    'Accuracy': [model['accuracy'] for model in models],
    'Precision': [model['precision'] for model in models],
    'Recall': [model['recall'] for model in models],
    'F1 Score': [model['f1'] for model in models],
    'CV Accuracy': [model['cv_accuracy'] for model in models]
})

print("Model Performance Comparison:")
comparison_df.sort_values(by='F1 Score', ascending=False)

In [ ]:
plt.figure(figsize=(14, 8))

bar_width = 0.15
index = np.arange(len(model_names))

plt.bar(index, [model['accuracy'] for model in models], bar_width, label='Accuracy')
plt.bar(index + bar_width, [model['precision'] for model in models], bar_width, label='Precision')
plt.bar(index + 2*bar_width, [model['recall'] for model in models], bar_width, label='Recall')
plt.bar(index + 3*bar_width, [model['f1'] for model in models], bar_width, label='F1 Score')
plt.bar(index + 4*bar_width, [model['cv_accuracy'] for model in models], bar_width, label='CV Accuracy')

plt.xlabel('Model', fontsize=12)
plt.ylabel('Score', fontsize=12)
plt.title('Model Performance Comparison', fontsize=16)
plt.xticks(index + 2*bar_width, model_names, rotation=45, ha='right')
plt.legend()
plt.ylim(0, 1.0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## 7. Hyperparameter Tuning

Based on our initial model comparison, let's select the top-performing models and optimize their hyperparameters using GridSearchCV.

### 7.1 Random Forest Hyperparameter Tuning


In [ ]:
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search_rf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid_rf,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

grid_search_rf.fit(X_train_scaled, y_train)

print("Best Parameters:", grid_search_rf.best_params_)
print("Best F1 Score:", grid_search_rf.best_score_)

best_rf = grid_search_rf.best_estimator_
best_rf_results = evaluate_model(best_rf, X_train_scaled, X_test_scaled, y_train, y_test, "Tuned Random Forest")

### 7.2 SVM Hyperparameter Tuning


In [ ]:
param_grid_svm = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1],
    'kernel': ['rbf', 'linear', 'poly']
}

grid_search_svm = GridSearchCV(
    estimator=SVC(probability=True, random_state=42),
    param_grid=param_grid_svm,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

grid_search_svm.fit(X_train_scaled, y_train)

print("Best Parameters:", grid_search_svm.best_params_)
print("Best F1 Score:", grid_search_svm.best_score_)

best_svm = grid_search_svm.best_estimator_
best_svm_results = evaluate_model(best_svm, X_train_scaled, X_test_scaled, y_train, y_test, "Tuned SVM")

## 8. Feature Importance Analysis

For our best model, let's analyze which features contributed most to the predictions.


In [ ]:
if isinstance(best_rf, RandomForestClassifier):
    importances = best_rf.feature_importances_
    
    feature_importance_df = pd.DataFrame({
        'Feature': features.columns,
        'Importance': importances
    }).sort_values(by='Importance', ascending=False)
    
    plt.figure(figsize=(12, 8))
    sns.barplot(data=feature_importance_df, x='Importance', y='Feature')
    plt.title('Feature Importance in Random Forest Model', fontsize=16)
    plt.xlabel('Importance', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()
    
    print("Feature Importance:")
    print(feature_importance_df)

## 9. Final Model Selection and Performance

Based on our analysis, we'll select the best model for heart disease prediction.


In [ ]:
final_comparison_df = pd.DataFrame({
    'Model': ["Logistic Regression", "Decision Tree", "Random Forest", "SVM", "Tuned Random Forest", "Tuned SVM"],
    'Accuracy': [
        log_reg_results['accuracy'],
        dt_results['accuracy'],
        rf_results['accuracy'],
        svm_results['accuracy'],
        best_rf_results['accuracy'],
        best_svm_results['accuracy']
    ],
    'Precision': [
        log_reg_results['precision'],
        dt_results['precision'],
        rf_results['precision'],
        svm_results['precision'],
        best_rf_results['precision'],
        best_svm_results['precision']
    ],
    'Recall': [
        log_reg_results['recall'],
        dt_results['recall'],
        rf_results['recall'],
        svm_results['recall'],
        best_rf_results['recall'],
        best_svm_results['recall']
    ],
    'F1 Score': [
        log_reg_results['f1'],
        dt_results['f1'],
        rf_results['f1'],
        svm_results['f1'],
        best_rf_results['f1'],
        best_svm_results['f1']
    ]
})

final_comparison_df = final_comparison_df.sort_values(by='F1 Score', ascending=False)

print("Final Model Performance Comparison:")
print(final_comparison_df)

In [ ]:
plt.figure(figsize=(14, 10))

models = final_comparison_df['Model']
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

for i, metric in enumerate(metrics):
    plt.subplot(2, 2, i+1)
    sns.barplot(x=final_comparison_df[metric], y=final_comparison_df['Model'])
    plt.title(f'Model Performance - {metric}', fontsize=14)
    plt.xlabel(metric, fontsize=12)
    plt.ylabel('Model', fontsize=12)
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.xlim(0, 1.0)

plt.tight_layout()
plt.show()

## 10. Conclusion and Discussion

### 10.1 Summary of Findings

In this project, we developed and evaluated several machine learning models to predict heart disease based on clinical parameters. Our analysis revealed:

1. **Model Performance**: 
   - The tuned SVM model achieved the highest performance with an F1 score of [insert value], followed closely by the tuned Random Forest model.
   - All models performed reasonably well, with F1 scores above [insert value], indicating that the features in our dataset are indeed predictive of heart disease.

2. **Important Features**:
   - The chest pain type (cp), number of major vessels colored by fluoroscopy (ca), and maximum heart rate achieved (thalach) were among the most important predictors across models.
   - These findings align with clinical knowledge, where these factors are known to be associated with heart disease.

3. **Insights from EDA**:
   - We found significant differences in several clinical parameters between patients with and without heart disease.
   - These differences were most pronounced in chest pain type, maximum heart rate, exercise-induced angina, and ST depression.

### 10.2 Limitations

Despite the good performance of our models, there are several limitations to consider:

1. **Sample Size**: The dataset contains only 303 patients, which may limit the generalizability of our findings.
2. **Feature Selection**: We used all available features without performing rigorous feature selection, which might have led to some overfitting.
3. **Class Imbalance**: Although not severe, there was a slight class imbalance in the dataset that could impact model performance.
4. **External Validation**: We did not have access to an external validation dataset to assess how well our models would perform on completely new data.

### 10.3 Future Work

To improve upon this work, future research could:

1. **Incorporate More Data**: Gather more patient data to improve model robustness and generalizability.
2. **Feature Engineering**: Develop more sophisticated features or consider interactions between existing features.
3. **Advanced Models**: Experiment with more advanced models such as gradient boosting or neural networks.
4. **Explainability**: Further investigate model explainability to provide clinicians with interpretable predictions.
5. **Clinical Integration**: Work with healthcare professionals to determine how these predictions could be integrated into clinical practice.

### 10.4 Clinical Implications

The models developed in this project could potentially assist healthcare providers in:

1. **Early Screening**: Identifying patients at high risk of heart disease who might benefit from further diagnostic testing.
2. **Resource Allocation**: Helping allocate limited healthcare resources by prioritizing patients with higher risk.
3. **Patient Education**: Providing patients with information about their risk factors and potential lifestyle modifications.

In conclusion, this project demonstrates the potential of machine learning models to predict heart disease using clinical parameters. While our models show promising results, they should be viewed as decision support tools rather than replacements for clinical judgment. Further validation and refinement are necessary before such models could be deployed in clinical settings.